In [3]:
from pyomo.environ import *
import numpy as np
import random
NG = 2 # Number of generators
NW = 1 # Number of wind farms
NL = 2 # Number of loads
#CG = np.array    Bo
#CL = np.array   Bo
#CW = np.array Bo
#B = Bo
#F = Bo
#Tamb = Bo/Ali
# Parameters related to the demand side
num_TCL = 1000
Tset_set = [21,21.5,22,22.5,23]
Tset = []
for i in range(num_TCL):
    s = random.sample(Tset_set,1)
    s = s[0]
    Tset.append(s)
delta = 
R_nom = 2
R_low = 0.8*R_nom
R_high = 1.2*R_nom
C_nom = 2
C_low = 0.8*C_nom
C_high = 1.2*C_nom
P_el_nom = 5.6
P_el_low = 0.8*P_el_nom
P_el_high = 1.2*P_el_nom
COP_nom = 2.5
COP_low = 0.8*COP_nom
COP_high = 1.2*COP_nom
R = np.random.uniform(R_low,R_high,num_TCL)
C = np.random.uniform(C_low,C_high,num_TCL)
P_el = np.random.uniform(P_el_low,P_el_high,num_TCL)
COP = np.random.uniform(COP_low,COP_high,num_TCL)

In [21]:
def VS(dt,Tamb,num_TCL,Tset,delta,R,C,P_el,COP):
    Tdown = np.zeros(num_TCL)
    Tup = np.zeros(num_TCL)
    T_ON = np.zeros(num_TCL)
    T_OFF = np.zeros(num_TCL)
    D = np.zeros(num_TCL)
    B = 0
    Pdown = 0
    Pup = 0
    Sdown = 0
    Sup = 0
    for i in range(0,num_TCL):
        Tdown[i] = Tset[i] - delta/2
        Tup[i] = Tset[i] + delta/2
        T_ON[i] = C[i]*R[i]*np.log((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]));
        T_OFF[i] = C[i]*R[i]*np.log((Tup[i] - Tamb)/(Tdown[i] - Tamb));
        D[i] = T_ON[i]/(T_ON[i] + T_OFF[i])
        print(T_ON[i])
        if ( Tamb < Tdown[i] and Tamb + R[i]*P_el[i]*COP[i] > Tup[i] ):
            B = B + P_el[i]*D[i]
            Pup = Pup + P_el[i]
            Sup = Sup + dt*P_el[i]*(1 - D[i])
    return [B,Pdown,Pup,Sdown,Sup]

In [22]:
ali = VS(0.25,-3,2,[22,22],1,[2,2],[3,3],[6,6],[2,2])
print(ali[2])

-6.591673732008658
-6.591673732008658
0
